In [ ]:
# Install pyfuzzy-toolbox with machine learning module
!pip install pyfuzzy-toolbox[ml] -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import fuzzy_systems as fs
from fuzzy_systems.learning import WangMendelLearning
from fuzzy_systems.inference import MamdaniSystem

%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['image.cmap']
np.random.seed(42)

In [ ]:
# ============================================================================
# Generate Problem Data
# ============================================================================

# True function
def true_function(x):
    """f(x) = -2x + 5"""
    return -2*x + 5

# Generate training data
n_samples = 50
X_train = np.linspace(0, 2*np.pi, n_samples).reshape(-1, 1)  # Shape (50, 1)
y_train = true_function(X_train).reshape(-1, 1)  # Shape (50, 1)
y_train += np.random.normal(0, 0.05, y_train.shape)

# Test data
X_test = np.linspace(1e-5, 2*np.pi, 200).reshape(-1, 1)  # Shape (200, 1)
y_true = true_function(X_test).flatten()


In [ ]:
# ============================================================================
# Create Mamdani Fuzzy System with 11 Partitions
# ============================================================================

system = MamdaniSystem(name='WangMendelLinearApprox')

# Domains
x_min, x_max = 0, 2*np.pi
y_min, y_max = y_train.min() - 0.1, y_train.max() + 0.1

# Add input and output variables 
system.add_input('x', (x_min, x_max))
system.add_output('y', (y_min, y_max))

#===============================================================================
#          USE THIS FOR DEFINING AUTOMATIC MFS
#===============================================================================

n_partitions = 5
system.add_auto_mfs('x',n_mfs=n_partitions,mf_type='triangular',label_prefix='In')
system.add_auto_mfs('y',n_mfs=n_partitions,mf_type='triangular',label_prefix='Out')


#===============================================================================
#          USE THIS FOR DEFINING CUSTOM MFS
#===============================================================================


# Create 11 overlapping triangular linguistic terms
# n_partitions = 11
# partition_names = [
#     'very_low', 'low_1', 'low_2', 'medium_low', 'medium_1', 'medium_2',
#     'medium_high', 'high_1', 'high_2', 'very_high_1', 'very_high_2'
# ]

# # Generate uniformly distributed partitions
# x_range = x_max - x_min
# step = x_range / (n_partitions - 1)

# for i, name in enumerate(partition_names):
#     center = x_min + i * step
#     left = max(x_min, center - step)
#     right = min(x_max, center + step)

#     if i == 0:
#         params = [x_min, x_min, center, center + step]
#         system.add_term('x', name, 'trapezoidal', params)
#     elif i == n_partitions - 1:
#         params = [center - step, center, x_max, x_max]
#         system.add_term('x', name, 'trapezoidal', params)
#     else:
#         params = [left, center, right]
#         system.add_term('x', name, 'triangular', params)

# print(f"Input 'x' configured with {n_partitions} linguistic terms")

# # Add output variable with 11 partitions
# system.add_output('y', (y_min, y_max))

# y_range = y_max - y_min
# y_step = y_range / (n_partitions - 1)

# output_names = [
#     'very_low', 'low_1', 'low_2', 'medium_low', 'medium_1', 'medium_2',
#     'medium_high', 'high_1', 'high_2', 'very_high_1', 'very_high_2'
# ]

# for i, name in enumerate(output_names):
#     center = y_min + i * y_step
#     left = max(y_min, center - y_step)
#     right = min(y_max, center + y_step)

#     if i == 0:
#         params = [y_min, y_min, center, center + y_step]
#         system.add_term('y', name, 'trapezoidal', params)
#     elif i == n_partitions - 1:
#         params = [center - y_step, center, y_max, y_max]
#         system.add_term('y', name, 'trapezoidal', params)
#     else:
#         params = [left, center, right]
#         system.add_term('y', name, 'triangular', params)

# print(f"Output 'y' configured with {n_partitions} linguistic terms")

fig,ax = system.plot_variables(return_axes=True)
fig.axes[0].get_legend().remove()
fig.axes[1].get_legend().remove()

In [ ]:
# ============================================================================
# Train with Wang-Mendel Algorithm
# ============================================================================

# Instantiate and train
wm = WangMendelLearning(system, X_train, y_train)
system_trained = wm.fit(verbose=True)

# Statistics
stats = wm.get_training_stats()
print(f'\nTraining Statistics:')
print(f'   • Candidate rules: {stats["candidate_rules"]}')
print(f'   • Final rules: {stats["final_rules"]}')
print(f'   • Conflicts resolved: {stats["conflicts_resolved"]}')

In [ ]:
# ============================================================================
# Make Predictions and Evaluate
# ============================================================================

# Prediction
y_pred = wm.predict(X_test).flatten()

# Metrics
mse = np.mean((y_pred - y_true)**2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(y_pred - y_true))
r2 = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_true.mean())**2))

print(f'\nPerformance Metrics:')
print(f'   • MSE (Mean Squared Error): {mse:.6f}')
print(f'   • RMSE (Root MSE): {rmse:.6f}')
print(f'   • MAE (Mean Absolute Error): {mae:.6f}')
print(f'   • R² Score: {r2:.6f}')

In [ ]:
# ============================================================================
# Visualization
# ============================================================================

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Subplot 1: Curve Comparison
ax1 = axes[0, 0]
ax1.plot(X_test, y_true, 'k-', linewidth=3, label='True Function', alpha=0.7, zorder=1)
ax1.plot(X_test, y_pred, 'b-', linewidth=2.5, label='Wang-Mendel Prediction', alpha=0.8, zorder=2)
ax1.scatter(X_train, y_train, color='red', s=80, zorder=5, 
           edgecolors='black', linewidths=1.5, label=f'Training Data (n={n_samples})', alpha=0.8)
ax1.set_xlabel('x', fontsize=12, fontweight='bold')
ax1.set_ylabel('y', fontsize=12, fontweight='bold')
ax1.set_title('Fuzzy Approximation vs True Function', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11, loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.text(0.02, 0.98, f'R² = {r2:.4f}\nRMSE = {rmse:.4f}', 
        transform=ax1.transAxes, fontsize=11, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Subplot 2: Prediction Error
ax2 = axes[0, 1]
error = y_pred - y_true
ax2.plot(X_test, error, 'r-', linewidth=2, alpha=0.7)
ax2.axhline(0, color='black', linestyle='--', linewidth=1.5, alpha=0.5)
ax2.fill_between(X_test.flatten(), error, 0, alpha=0.3, color='red')
ax2.set_xlabel('x', fontsize=12, fontweight='bold')
ax2.set_ylabel('Error (Prediction - Real)', fontsize=12, fontweight='bold')
ax2.set_title('Prediction Error', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.text(0.02, 0.98, f'MAE = {mae:.4f}\nMax Error = {np.abs(error).max():.4f}', 
        transform=ax2.transAxes, fontsize=11, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.5))

# Subplot 3: Error Histogram
ax3 = axes[1, 0]
ax3.hist(error, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
ax3.axvline(0, color='red', linestyle='--', linewidth=2, label='Zero Error')
ax3.set_xlabel('Error', fontsize=12, fontweight='bold')
ax3.set_ylabel('Frequency', fontsize=12, fontweight='bold')
ax3.set_title('Error Distribution', fontsize=14, fontweight='bold')
ax3.legend(fontsize=11)
ax3.grid(True, alpha=0.3, axis='y')

# Subplot 4: Prediction vs Real Scatter
ax4 = axes[1, 1]
ax4.scatter(y_true, y_pred, alpha=0.5, s=30, edgecolors='black', linewidths=0.5)
ax4.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 
        'r--', linewidth=2, label='Perfect Prediction', zorder=10)
ax4.set_xlabel('True Value', fontsize=12, fontweight='bold')
ax4.set_ylabel('Predicted Value', fontsize=12, fontweight='bold')
ax4.set_title('Prediction vs True', fontsize=14, fontweight='bold')
ax4.legend(fontsize=11)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# Visualize Learned Rules
# ============================================================================

fig,ax = wm.system.plot_rule_matrix(figsize=(8,4))

# ============================================================================
# Customizing the visualization
# ============================================================================

fig.axes[0].set_title('Rule Visualization', fontsize=14, fontweight='bold', pad=40)
fig.axes[1].remove()
ax.images[0].set_cmap('ocean')